In [ ]:
%pip install transformers datasets trl
%pip install fsspec==2023.9.2
%pip install datasets
%pip install -U datasets

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
import pandas as pd
import random
from itertools import product

splits = {'train': 'plain_text/train-00000-of-00001.parquet', 'test': 'plain_text/test-00000-of-00001.parquet', 'unsupervised': 'plain_text/unsupervised-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/stanfordnlp/imdb/" + splits["train"])
positive_comments = df[df['label'] == 1]['text'].tolist()
negative_comments = df[df['label'] == 0]['text'].tolist()

num_samples = 1000
sampled_positive_comments = random.sample(positive_comments, num_samples)
sampled_negative_comments = random.sample(negative_comments, num_samples)

comment_pairs = list(product(sampled_positive_comments, sampled_negative_comments))
print(f"Number of comment pairs: {len(comment_pairs)}")

Number of comment pairs: 1000000


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments
from datasets import Dataset
import torch

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-cased')

pairs_df = pd.DataFrame(comment_pairs, columns=['positive_comment', 'negative_comment'])
dataset = Dataset.from_pandas(pairs_df)

def preprocess_function(examples):
    chosen_inputs = tokenizer(examples['positive_comment'], truncation=True, padding='max_length', max_length=512)
    rejected_inputs = tokenizer(examples['negative_comment'], truncation=True, padding='max_length', max_length=512)
    return {
        'input_ids_chosen': chosen_inputs['input_ids'],
        'attention_mask_chosen': chosen_inputs['attention_mask'],
        'input_ids_rejected': rejected_inputs['input_ids'],
        'attention_mask_rejected': rejected_inputs['attention_mask']
    }

tokenized_dataset = dataset.map(preprocess_function, batched=True)
train_dataset = tokenized_dataset.shuffle().select(range(1000))

for column in ['input_ids_chosen', 'attention_mask_chosen', 'input_ids_rejected', 'attention_mask_rejected']:
    assert column in train_dataset.features


/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
Map: 100%|██████████| 1000000/1000000 [14:14<00:00, 1170.57 examples/s]


In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch

model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-cased', num_labels=1)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=8,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    remove_unused_columns=False
)

def data_collator(features):
    batch = {
        'input_ids': [],
        'attention_mask': [],
        'labels': []
    }

    for feature in features:
        batch['input_ids'].append(feature['input_ids_chosen'])
        batch['attention_mask'].append(feature['attention_mask_chosen'])
        batch['labels'].append(1.0)

        batch['input_ids'].append(feature['input_ids_rejected'])
        batch['attention_mask'].append(feature['attention_mask_rejected'])
        batch['labels'].append(0.0)

    batch = {k: torch.tensor(v) for k, v in batch.items()}

    batch['input_ids'] = batch['input_ids'].long()
    batch['attention_mask'] = batch['attention_mask'].long()
    batch['labels'] = batch['labels'].float()

    return batch

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator
)

trainer.train()

model.save_pretrained('./reward_model')
tokenizer.save_pretrained('./reward_model')

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.

 99%|█████████▉| 124/125 [00:33<00:00,  3.82it/s]
                                        3.87it/s]
100%|██████████| 125/125 [00:38<00:00,  3.24it/s]

{'train_runtime': 38.5608, 'train_samples_per_second': 25.933, 'train_steps_per_second': 3.242, 'train_loss': 0.1842788391113281, 'epoch': 1.0}


('./reward_model/tokenizer_config.json',
 './reward_model/special_tokens_map.json',
 './reward_model/vocab.txt',
 './reward_model/added_tokens.json',
 './reward_model/tokenizer.json')

In [ ]:
reward_model = AutoModelForSequenceClassification.from_pretrained('./reward_model')
reward_tokenizer = AutoTokenizer.from_pretrained('./reward_model')

In [ ]:
import random

def create_prompts(df, num_prompts=100, min_tokens=5, max_tokens=15, max_length=512):
    prompts = []
    for text in df['text'].sample(num_prompts):
        tokens = tokenizer.tokenize(text)
        if len(tokens) > min_tokens:
            truncated_tokens = tokens[:random.randint(min_tokens, max_tokens)]
            truncated_tokens = truncated_tokens[:max_length]  # Ensure the sequence is within the max length
            prompts.append(tokenizer.convert_tokens_to_string(truncated_tokens))
    return prompts

train_prompts = create_prompts(df, num_prompts=100)

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW
from tqdm.auto import tqdm
import torch
import torch.nn.functional as F

gpt2_model = GPT2LMHeadModel.from_pretrained('lvwerra/gpt2-imdb')
ref_model = GPT2LMHeadModel.from_pretrained('lvwerra/gpt2-imdb')
optimizer = AdamW(gpt2_model.parameters(), lr=1e-5)
tokenizer = GPT2Tokenizer.from_pretrained('lvwerra/gpt2-imdb')

tokenizer.add_special_tokens({'pad_token': '[PAD]'})
gpt2_model.resize_token_embeddings(len(tokenizer))
ref_model.resize_token_embeddings(len(tokenizer))

reward_model = AutoModelForSequenceClassification.from_pretrained('./reward_model')
reward_tokenizer = AutoTokenizer.from_pretrained('./reward_model')

theta_init = gpt2_model.state_dict()
theta_sft = gpt2_model.state_dict()


In [ ]:
def compute_reward(prompt, generated_text):
    prompt_inputs = reward_tokenizer(prompt, return_tensors="pt", truncation=True, padding=True)
    generated_inputs = reward_tokenizer(generated_text, return_tensors="pt", truncation=True, padding=True)

    with torch.no_grad():
        generated_output = reward_model(**generated_inputs)

    reward = torch.sigmoid(generated_output.logits).mean().item()
    return reward

def slurp(theta_init, thetas, lambd):
    result = {}
    for key in theta_init.keys():
        result[key] = (1 - lambd) * theta_init[key]
        for theta in thetas:
            result[key] += (lambd / len(thetas)) * theta[key]
    return result

def evaluate_kl_divergence(output, ref_model_output):
    kl_divergence = F.kl_div(output.log_softmax(dim=-1), ref_model_output.softmax(dim=-1), reduction='batchmean')
    return kl_divergence.mean()


In [ ]:
def create_prompts(df, num_prompts=100, min_tokens=5, max_tokens=15, max_length=512):
    prompts = []
    for text in df['text'].sample(num_prompts):
        tokens = tokenizer.tokenize(text)
        if len(tokens) > min_tokens:
            truncated_tokens = tokens[:random.randint(min_tokens, max_tokens)]
            truncated_tokens = truncated_tokens[:max_length]
            prompts.append(tokenizer.convert_tokens_to_string(truncated_tokens))
    return prompts

train_prompts = create_prompts(df, num_prompts=100)


In [ ]:
I = 2
M = 2
T = 100
mu = 0.01
nu = 0.5

theta_ema = theta_init.copy()

for i in tqdm(range(I), desc="Iterations", position=0, leave=True):
    for m in tqdm(range(M), desc="Models", position=1, leave=True):
        theta_m = gpt2_model.state_dict()

        for t in tqdm(range(T), desc="Training Steps", position=2, leave=True):
            optimizer.zero_grad()
            inputs = tokenizer(train_prompts, return_tensors="pt", padding=True, truncation=True)
            inputs = {key: value.to(gpt2_model.device) for key, value in inputs.items()}
            outputs = gpt2_model(**inputs)
            ref_outputs = ref_model(**inputs)
            generated_texts = tokenizer.batch_decode(outputs.logits.argmax(dim=-1), skip_special_tokens=True)

            rewards = [compute_reward(prompt, text) for prompt, text in zip(train_prompts, generated_texts)]
            rewards = torch.tensor(rewards).to(gpt2_model.device)
            kl_divergence = evaluate_kl_divergence(outputs.logits, ref_outputs.logits)
            kl_regularized_reward = rewards - nu * kl_divergence
            loss = -kl_regularized_reward.mean()
            loss.backward()
            optimizer.step()

            for key in theta_ema.keys():
                theta_ema[key] = (1 - mu) * theta_ema[key] + mu * gpt2_model.state_dict()[key]

        for key in theta_m.keys():
            theta_m[key] = gpt2_model.state_dict()[key]

    theta_slerp = slurp(theta_init, [theta_m] * M, lambd=0.5)
    for key in theta_init.keys():
        theta_init[key] = (1 - nu) * theta_init[key] + nu * theta_slerp[key]

final_weights = {key: (1 - nu) * theta_sft[key] + nu * theta_slerp[key] for key in theta_sft.keys()}
gpt2_model.load_state_dict(final_weights)


Models:   0%|          | 0/2 [00:00<?, ?it/s]

Training Steps:   0%|          | 0/100 [00:00<?, ?it/s]

Training Steps:   1%|          | 1/100 [00:19<32:23, 19.63s/it]

Training Steps:   2%|▏         | 2/100 [00:39<31:58, 19.57s/it]

Training Steps:   3%|▎         | 3/100 [00:58<31:24, 19.43s/it]

Training Steps:   4%|▍         | 4/100 [01:17<30:53, 19.31s/it]

Training Steps:   5%|▌         | 5/100 [01:37<30:42, 19.39s/it]

Training Steps:   6%|▌         | 6/100 [01:56<30:15, 19.32s/it]

Training Steps:   7%|▋         | 7/100 [02:15<29:45, 19.20s/it]

Training Steps:   8%|▊         | 8/100 [02:34<29:14, 19.07s/it]

Training Steps:   9%|▉         | 9/100 [02:52<28:40, 18.90s/it]

Training Steps:  10%|█         | 10/100 [03:11<28:16, 18.84s/it]

Training Steps:  11%|█         | 11/100 [03:29<27:53, 18.80s/it]

Training Steps:  12%|█▏        | 12/100 [03:48<27:20, 18.64s/it]

Training Steps:  13%|█▎        | 13/100 [04:06<26:59, 18.61s/it]

Training Steps:  14%|█▍        | 14/100 [04:25<

<All keys matched successfully>

In [ ]:
import torch.nn.functional as F

test_df = pd.read_parquet("hf://datasets/stanfordnlp/imdb/" + splits["test"])
test_prompts = create_prompts(test_df, num_prompts=100)

generated_responses = []
rewards = []
kl_divergences = []

def pad_or_truncate(tensor, max_length):
    if tensor.size(1) > max_length:
        return tensor[:, :max_length]
    elif tensor.size(1) < max_length:
        pad_size = max_length - tensor.size(1)
        return torch.nn.functional.pad(tensor, (0, 0, 0, pad_size), mode='constant', value=tokenizer.pad_token_id)
    return tensor

for prompt in tqdm(test_prompts, desc="Evaluation", position=0, leave=True):
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
    inputs = {key: value.to(gpt2_model.device) for key, value in inputs.items()}
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    output = gpt2_model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=input_ids.shape[-1] + 50,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id
    )

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    generated_responses.append(generated_text)

    reward = compute_reward(prompt, generated_text)
    rewards.append(reward)

    with torch.no_grad():
        ref_outputs = ref_model(input_ids, attention_mask=attention_mask)

    generated_inputs = tokenizer(generated_text, return_tensors="pt", padding=True, truncation=True).to(gpt2_model.device)
    generated_outputs = gpt2_model(**generated_inputs)

    max_length = max(generated_outputs.logits.size(1), ref_outputs.logits.size(1))
    generated_logits_padded = pad_or_truncate(generated_outputs.logits, max_length)
    ref_logits_padded = pad_or_truncate(ref_outputs.logits, max_length)

    kl_divergence = F.kl_div(generated_logits_padded.log_softmax(dim=-1), ref_logits_padded.softmax(dim=-1), reduction='batchmean')
    kl_divergences.append(kl_divergence.mean().item())

avg_reward = sum(rewards) / len(rewards)
avg_kl_divergence = sum(kl_divergences) / len(kl_divergences)

print(f"Average Reward: {avg_reward}, Average KL Divergence: {avg_kl_divergence}")



Evaluation: 100%|██████████| 100/100 [02:29<00:00,  1.50s/it]

Average Reward: 0.6583488184213638, Average KL Divergence: 439.11874591827393


In [ ]:
import matplotlib.pyplot as plt

def run_warp_with_mu(mu_value):
    theta_init = gpt2_model.state_dict()
    theta_sft = gpt2_model.state_dict()
    theta_ema = theta_init.copy()

    for i in range(I):
        for m in range(M):
            theta_m = gpt2_model.state_dict()
            for t in range(T):
                optimizer.zero_grad()
                inputs = tokenizer(train_prompts, return_tensors="pt", padding=True, truncation=True)
                inputs = {key: value.to(gpt2_model.device) for key, value in inputs.items()}
                outputs = gpt2_model(**inputs)
                ref_outputs = ref_model(**inputs)
                generated_texts = tokenizer.batch_decode(outputs.logits.argmax(dim=-1), skip_special_tokens=True)

                rewards = [compute_reward(prompt, text) for prompt, text in zip(train_prompts, generated_texts)]
                rewards = torch.tensor(rewards).to(gpt2_model.device)
                kl_divergence = evaluate_kl_divergence(outputs.logits.float(), ref_outputs.logits.float())
                kl_regularized_reward = rewards - nu * kl_divergence
                loss = -kl_regularized_reward.mean()
                loss.backward()
                optimizer.step()

                for key in theta_ema.keys():
                    theta_ema[key] = (1 - mu_value) * theta_ema[key] + mu_value * gpt2_model.state_dict()[key]
            for key in theta_m.keys():
                theta_m[key] = gpt2_model.state_dict()[key]
        theta_slerp = slurp(theta_init, [theta_m] * M, lambd=0.5)
        for key in theta_init.keys():
            theta_init[key] = (1 - nu) * theta_init[key] + nu * theta_slerp[key]

    final_weights = {key: (1 - nu) * theta_sft[key] + nu * theta_slerp[key] for key in theta_sft.keys()}
    gpt2_model.load_state_dict(final_weights)

    generated_responses = []
    rewards = []
    kl_divergences = []

    for prompt in test_prompts:
        inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
        inputs = {key: value.to(gpt2_model.device) for key, value in inputs.items()}
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']

        output = gpt2_model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_length=input_ids.shape[-1] + 50,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id
        )

        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        generated_responses.append(generated_text)

        reward = compute_reward(prompt, generated_text)
        rewards.append(reward)

        ref_outputs = ref_model(input_ids)
        generated_inputs = tokenizer(generated_text, return_tensors="pt", padding=True, truncation=True).to(gpt2_model.device)
        generated_outputs = gpt2_model(**generated_inputs)

        max_length = max(generated_outputs.logits.size(1), ref_outputs.logits.size(1))
        generated_logits_padded = pad_or_truncate(generated_outputs.logits, max_length)
        ref_logits_padded = pad_or_truncate(ref_outputs.logits, max_length)

        kl_divergence = F.kl_div(generated_logits_padded.log_softmax(dim=-1), ref_logits_padded.softmax(dim=-1), reduction='batchmean')
        kl_divergences.append(kl_divergence.mean().item())

    avg_reward = sum(rewards) / len(rewards)
    avg_kl_divergence = sum(kl_divergences) / len(kl_divergences)

    return avg_reward, avg_kl_divergence

mu_values = [0.01, 0.05, 0.1]
results = []

for mu in mu_values:
    avg_reward, avg_kl_divergence = run_warp_with_mu(mu)
    results.append((mu, avg_reward, avg_kl_divergence))
    print(f"mu: {mu}, Average Reward: {avg_reward}, Average KL Divergence: {avg_kl_divergence}")

mus, rewards, kls = zip(*results)

fig, ax1 = plt.subplots()

color = 'tab:blue'
ax1.set_xlabel('mu')
ax1.set_ylabel('Average Reward', color=color)
ax1.plot(mus, rewards, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()
color = 'tab:red'
ax2.set_ylabel('Average KL Divergence', color=color)
ax2.plot(mus, kls, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()
plt.title('Average Reward and KL Divergence vs mu')
plt.show()

## Вывод

**Цель**

Цель проекта заключалась в реализации алгоритма WARP (Weighted Approximate Rank Pairwise) и его оценке с использованием модели наград, обученной на датасете IMDb. Мы стремились измерить среднюю награду и дивергенцию Кульбака-Лейблера (KL), а также визуализировать влияние различных гиперпараметров на производительность модели.

**Шаги реализации**

1.Подготовка данных:

Загрузили датасет IMDb.
Создали пары положительных и отрицательных комментариев из обучающей подвыборки.
Обучили модель наград, используя distilbert-base-cased, на этих парах.
Реализация алгоритма WARP:

Инициализировали модель GPT-2 (gpt2-imdb) и модель наград.
Реализовали алгоритм WARP с итерациями по разным моделям и шагам обучения.
Использовали SLERP (Spherical Linear Interpolation) для объединения весов моделей.

2.Оценка:

Сгенерировали 100 запросов из тестовой подвыборки датасета IMDb.
Измерили среднюю награду и дивергенцию KL для сгенерированных выводов.
Реализовали дополнение и усечение для обеспечения совпадения размеров тензоров при вычислении дивергенции KL.

3.Настройка гиперпараметров:

Выбрали mu как гиперпараметр для изменения.
Запустили алгоритм WARP с тремя разными значениями mu (0.01, 0.05 и 0.1).

**Результаты**

Средняя награда и дивергенция KL:

При mu=0.01: Средняя награда: 0.658, Средняя дивергенция KL: 439.119
При mu=0.05: (Не удалось получить в указанные сроки ввиду ограничения вычислительных мощностей)
При mu=0.1: (Не удалось получить в указанные сроки ввиду ограничения вычислительных мощностей)


Проблемы и возможные причины
Высокая дивергенция KL:

Одной из основных проблем была высокая дивергенция KL, что указывало на значительное расхождение между выходными данными модели и референтной модели.


**Выводы**

Алгоритм WARP успешно изменяет веса модели, но высокая дивергенция KL указывает на необходимость дальнейшей настройки и оценки.